# Ejercicio 11: Web Scraping

## Objetivo de la práctica

El objetivo de este ejercicio es construir un web scraper que recoja datos de un website.

### Parte 0: Planificar
1. Identificar los datos que quieres obtener.
2. Elegir el sitio web objetivo.
3. Planificar la estructura del corpus.

## Parte 1: Entender el sitio web objetivo

- Analizar la estructura de la página web a ser analizada.
- Identificar los elementos HTML que contienen los datos bsuscados.

In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [17]:
from bs4 import BeautifulSoup
import os

In [18]:
ruta_archivo = "/kaggle/input/html2-1/unstuffed-cabbage-roll.html"

In [19]:
# Usamos 'utf-8' para evitar problemas con tildes o caracteres especiales
with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
    contenido_html = archivo.read()

In [20]:
soup = BeautifulSoup(contenido_html, 'html.parser')

In [21]:
# Extracting the recipe title
title = soup.find("title")
title.string

'Unstuffed Cabbage Roll Recipe'

In [22]:
#score
#description
#prep time
#cooking time
#total time
#servings
#ingredients
#directtions
#nutritions facts
#Reviews

In [23]:
score=soup.find("div", {"id":"mm-recipes-review-bar__rating_1-0"})
score.string

'4.6'

In [24]:
description=soup.find("p", {"class":"article-subheading text-utility-300"})
description.string

"This is an easy casserole made with ground beef, cabbage, garlic, and tomatoes. My kids don't even like cabbage, but they love this dish! Serve with rice for a comforting weeknight dinner. Also, the longer it stands the better it tastes!"

In [25]:
prep_time=soup.find("div", {"class":"mm-recipes-details__value"})
prep_time.string

'15 mins'

## Parte 2: Obtener los datos deseados

* Buscar dentro del contenido HTML y extraer la información.

In [26]:
def parsearReceta(html_doc_path):
    with open(html_doc_path, 'r', encoding='utf-8') as archivo:
        html_doc = archivo.read()

    soup = BeautifulSoup(html_doc, 'html.parser')
    
    score=soup.find("div", {"id":"mm-recipes-review-bar__rating_1-0"}).string
    prep_time=soup.find("div", {"class":"mm-recipes-details__value"}).string
    description=soup.find("p", {"class":"article-subheading text-utility-300"}).string
    title = soup.find("title").string
    return {"score":score,"prep_time":prep_time, "description":description, "title":title}

In [27]:
ruta_receta2 = "/kaggle/input/receta-2/chicken-cobbler-recipe-7966464.html"

In [28]:
receta2 = parsearReceta(ruta_receta2)
receta2

{'score': '4.3',
 'prep_time': '5 mins',
 'description': "This viral TikTok chicken cobbler made with Red Lobster biscuit mix is incredibly easy to put together. We love this recipe from Matthew Bounds, aka @yourbarefootneighbor. Try it yourself to find out what everyone's raving about!",
 'title': 'Chicken Cobbler Recipe'}

## Parte 3: Obtener enlaces relacionados
* Encontrar links a otras recetas para completar el corpus

In [29]:
# Find all the links to other recipes
recipe_links = soup.find_all("a", href=True)

# Filter and print only the links that are likely to be recipes
recipe_urls = []
for link in recipe_links:
    href = link['href']
    if "recipe" in href:
        recipe_urls.append(href)

# Print the recipe URLs
print("Linked Recipes:")
for url in recipe_urls:
    print(url)

Linked Recipes:
https://www.allrecipes.com/authentication/login?regSource=3675&relativeRedirectUrl=%2Frecipe%2F235997%2Funstuffed-cabbage-roll%2F
/account/add-recipe
https://www.myrecipes.com/favorites
https://support.people.inc/hc/en-us/categories/360003648613-Allrecipes
https://www.allrecipes.com/authentication/logout?relativeRedirectUrl=%2Frecipe%2F235997%2Funstuffed-cabbage-roll%2F
https://www.allrecipes.com/recipes/17562/dinner/
https://www.allrecipes.com/recipes/17057/everyday-cooking/more-meal-ideas/5-ingredients/main-dishes/
https://www.allrecipes.com/recipes/15436/everyday-cooking/one-pot-meals/
https://www.allrecipes.com/recipes/1947/everyday-cooking/quick-and-easy/
https://www.allrecipes.com/recipes/455/everyday-cooking/more-meal-ideas/30-minute-meals/
https://www.allrecipes.com/recipes/17889/everyday-cooking/family-friendly/family-dinners/
https://www.allrecipes.com/recipes/94/soups-stews-and-chili/
https://www.allrecipes.com/recipes/16099/everyday-cooking/comfort-food/
htt

## Parte 4: Hacer RAG con las recetas obtenidas
* Una vez que se ha construido el corpus, implementar y desplegar RAG para realizar búsquedas en el corpus

In [31]:
import requests
from bs4 import BeautifulSoup
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from langchain_community.llms import OpenAI

def parsearReceta_desde_web(html_content, url):
    soup = BeautifulSoup(html_content, 'html.parser')
    try:
        score_div = soup.find("div", {"id": lambda x: x and "rating" in x}) # Selector más flexible
        score = score_div.get_text(strip=True) if score_div else "N/A"
        
        prep_div = soup.find("div", {"class": "mm-recipes-details__value"})
        prep_time = prep_div.get_text(strip=True) if prep_div else "N/A"
        
        desc_p = soup.find("p", {"class": "article-subheading"})
        description = desc_p.get_text(strip=True) if desc_p else ""
        
        title_tag = soup.find("title")
        title = title_tag.get_text(strip=True) if title_tag else "Sin título"
        
        # Formateamos el texto para que la IA lo entienda mejor
        texto_final = f"RECETA: {title}\nDESCRIPCIÓN: {description}\nTIEMPO: {prep_time}\nPUNTUACIÓN: {score}"
        
        return {"texto": texto_final, "titulo": title, "url": url}
    except Exception as e:
        print(f"Error parseando {url}: {e}")
        return None

In [32]:
docs_para_rag = []

print(f"Procesando {len(recipe_urls)} recetas...")

for url in recipe_urls:
    try:
        # Descargamos el contenido de la URL
        response = requests.get(url, timeout=10)
        
        if response.status_code == 200:
            # Pasamos el contenido crudo a tu parser
            datos = parsearReceta_desde_web(response.text, url)
            
            if datos:
                # Creamos el objeto Documento para LangChain
                doc = Document(
                    page_content=datos["texto"],
                    metadata={"source": datos["url"], "title": datos["titulo"]}
                )
                docs_para_rag.append(doc)
                print(f"Procesada: {datos['titulo']}")
        else:
            print(f"Error descargando {url}: Status {response.status_code}")
            
    except Exception as e:
        print(f"Error de conexión con {url}")

print(f"\nTotal documentos listos para RAG: {len(docs_para_rag)}")

Procesando 203 recetas...
Procesada: Sign in to Allrecipes
Error de conexión con /account/add-recipe
Procesada: Sign in to MyRecipes
Error descargando https://support.people.inc/hc/en-us/categories/360003648613-Allrecipes: Status 403
Procesada: Unstuffed Cabbage Roll Recipe
Procesada: Dinner Recipes
Procesada: 5 Ingredient Main Dish Recipes
Procesada: One-Pot Meal Recipes
Procesada: Quick and Easy Recipes
Procesada: 30-Minute Meal Recipes
Procesada: Family Dinner Ideas & Recipes
Procesada: Soups, Stews and Chili Recipes
Procesada: Comfort Food Recipes
Procesada: Main Dishes
Procesada: Sheet Pan Dinner Recipes
Procesada: Dinner Recipes
Procesada: Recipes A-Z | Allrecipes.com
Procesada: Breakfast and Brunch Recipes
Procesada: Lunch Recipes
Procesada: Healthy Recipes
Procesada: Appetizers and Snacks
Procesada: Salad Recipes
Procesada: Side Dish Recipes
Procesada: Soup Recipes
Procesada: Bread Recipes
Procesada: Drinks Recipes
Procesada: Dessert Recipes
Procesada: Recipes A-Z | Allrecipes.

In [33]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_db = Chroma.from_documents(
    documents=docs_para_rag,
    embedding=embedding_model
)


retriever = vector_db.as_retriever(search_kwargs={"k": 2}) # Traer las 2 recetas más relevantes


def buscar_receta(pregunta):
    docs_rellevantes = retriever.invoke(pregunta)
    print(f"\n🔎 Pregunta: {pregunta}")
    print("-" * 30)
    
    for i, doc in enumerate(docs_rellevantes):
        print(f"Top {i+1} Recomendación:")
        print(f"Titulo: {doc.metadata['title']}")
        print(f"Link: {doc.metadata['source']}")
        print(f"Contenido parcial: {doc.page_content[:150]}...") # Primeros 150 caracteres
        print("-" * 10)

# --- PRUEBA DEL SISTEMA ---
buscar_receta("¿Hay alguna receta que sea rápida de hacer?")
buscar_receta("Quiero cocinar algo dulce o postre")

/tmp/ipykernel_258/2050075924.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
2026-01-23 01:01:11.799992: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769130072.291312     258 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769130072.443973     258 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attemptin

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


🔎 Pregunta: ¿Hay alguna receta que sea rápida de hacer?
------------------------------
Top 1 Recomendación:
Titulo: Fast Food
Link: https://www.allrecipes.com/fast-food-8672832
Contenido parcial: RECETA: Fast Food
DESCRIPCIÓN: 
TIEMPO: N/A
PUNTUACIÓN: N/A...
----------
Top 2 Recomendación:
Titulo: Fast Food
Link: https://www.allrecipes.com/fast-food-8672832
Contenido parcial: RECETA: Fast Food
DESCRIPCIÓN: 
TIEMPO: N/A
PUNTUACIÓN: N/A...
----------

🔎 Pregunta: Quiero cocinar algo dulce o postre
------------------------------
Top 1 Recomendación:
Titulo: Bread Recipes
Link: https://www.allrecipes.com/recipes/156/bread/
Contenido parcial: RECETA: Bread Recipes
DESCRIPCIÓN: 
TIEMPO: N/A
PUNTUACIÓN: ...
----------
Top 2 Recomendación:
Titulo: Bread Recipes
Link: https://www.allrecipes.com/recipes/156/bread/
Contenido parcial: RECETA: Bread Recipes
DESCRIPCIÓN: 
TIEMPO: N/A
PUNTUACIÓN: ...
----------
